Preparing to submit wold stranded samples....

This got a little messy because a couple libraries needed a top up.

In [1]:
import os
import sys
import requests
import pandas
import paramiko
import re
import json
from IPython import display
from pathlib import Path
import configparser

In [2]:
from curation_common import *
from encoded_client.encoded import DCCValidator

In [3]:
from encoded_client.encoded import Document
from encoded_client.submission import run_aws_cp
from htsworkflow.util.api import (
    add_auth_options,
    make_auth_from_opts,
    HtswApi,
)

In [4]:
ls /etc/ssl/certs/ca-certificates.crt

/etc/ssl/certs/ca-certificates.crt


In [5]:
config = configparser.ConfigParser()
config.read([os.path.expanduser('~/.htsworkflow.ini'),
             '/etc/htsworkflow.ini'
             ])

SECTION = 'sequence_archive'
if config.has_section(SECTION):
    apiid = config.get(SECTION, 'apiid')
    apikey = config.get(SECTION, 'apikey')
    apihost = config.get(SECTION, 'host')

auth = {'apiid': apiid, 'apikey': apikey }
htsw = HtswApi(apihost, auth)

In [6]:
# live server & control file
server = ENCODED('www.encodeproject.org')
engine=None
#engine='odf'

# test server & datafile
#server = ENCODED('test.encodedcc.org')
#spreadsheet_name = os.path.expanduser('~diane/woldlab/ENCODE/C1-encode3-limb-2017-testserver.ods')

server.load_netrc()
validator = DCCValidator(server)

In [7]:
award = 'UM1HG009443'

# Lookup biosample ontologies

Lookup any biosample ontologies that are already present

In [17]:
clinical_to_disease_map = {
    'MCI + no CI': ['DOID:0080832'],
    'NINCDS PROB AD': ['DOID:10652'],
    'NINCDS POSS AD': ['DOID:10652', 'HP:0100543'],
    'MCI + CI': ['HP:0100543'],
    'NCI': None,
}

In [20]:
def fix_disease_term_id(spreadhseet_name, dry_run=True):
    print(spreadsheet_name)
    biosample = pandas.read_excel(spreadsheet_name, sheet_name='Biosample', header=0, engine=engine)
    count = 0

    for i, row in biosample.iterrows():
        if not pandas.isnull(row.accession) and row.accession.startswith('E'):
            clinical_status = row["clinical_status:skip"]
            expected_disease_id = clinical_to_disease_map[clinical_status]
            obj = server.get_json(row.accession)
            disease_id = obj.get("disease_term_id")
            if disease_id != expected_disease_id:
                count += 1
                payload = {"disease_term_id": expected_disease_id}
                if dry_run:
                    print(obj["@id"], payload)
                else:
                    print(server.patch_json(obj["@id"], payload))
    print("change", count)
    return biosample

In [24]:
spreadsheet_name = Path('~/woldlab/ENCODE/stranded-24038-24061-rush-resubmit.xlsx').expanduser()
biosample = fix_disease_term_id(spreadsheet_name, dry_run=True)

/home/diane/woldlab/ENCODE/stranded-24038-24061-rush-resubmit.xlsx
/biosamples/ENCBS644WSW/ {'disease_term_id': ['DOID:10652']}
/biosamples/ENCBS436WHA/ {'disease_term_id': ['DOID:10652']}
/biosamples/ENCBS481WHG/ {'disease_term_id': ['DOID:10652']}
/biosamples/ENCBS660AUE/ {'disease_term_id': ['DOID:10652']}
/biosamples/ENCBS422TMB/ {'disease_term_id': ['DOID:10652']}
/biosamples/ENCBS675FWG/ {'disease_term_id': ['DOID:10652']}
/biosamples/ENCBS431HLD/ {'disease_term_id': ['DOID:0080832']}
/biosamples/ENCBS542OFI/ {'disease_term_id': ['DOID:10652', 'HP:0100543']}
/biosamples/ENCBS958WOW/ {'disease_term_id': ['DOID:0080832']}
/biosamples/ENCBS928RYN/ {'disease_term_id': ['DOID:10652']}
/biosamples/ENCBS839OMR/ {'disease_term_id': ['DOID:10652', 'HP:0100543']}
/biosamples/ENCBS048EXC/ {'disease_term_id': ['DOID:10652']}
change 12


In [25]:
spreadsheet_name = Path('~/woldlab/ENCODE/stranded-24196-24219-rush-resubmit.xlsx').expanduser()
biosample = fix_disease_term_id(spreadsheet_name, dry_run=True)

/home/diane/woldlab/ENCODE/stranded-24196-24219-rush-resubmit.xlsx
{'status': 'success', '@type': ['result'], '@graph': [{'accession': 'ENCBS190LXS', 'aliases': ['barbara-wold:ENC4_cDNA673'], 'schema_version': '26', 'status': 'in progress', 'lab': '/labs/barbara-wold/', 'award': '/awards/UM1HG009443/', 'date_created': '2022-03-28T20:24:50.793786+00:00', 'submitted_by': '/users/bc5b62f7-ce28-4a1e-b6b3-81c9c5a86d7a/', 'documents': [], 'references': [], 'source': '/sources/rush-university/', 'biosample_ontology': '/biosample-types/tissue_UBERON_0006483/', 'genetic_modifications': [], 'alternate_accessions': [], 'description': 'human_brain_Rush ID_E3253149_BS96_ENC4_cDNA673', 'treatments': [], 'dbxrefs': [], 'donor': '/human-donors/ENCDO877NVF/', 'organism': '/organisms/human/', 'internal_tags': ['RushAD'], 'disease_term_id': ['DOID:0080832'], '@id': '/biosamples/ENCBS190LXS/', '@type': ['Biosample', 'Item'], 'uuid': '3fa91232-1624-432f-8fe0-17fc63e67ee5', 'sex': 'male', 'age': '90 or abov

{'status': 'success', '@type': ['result'], '@graph': [{'accession': 'ENCBS745ELX', 'aliases': ['barbara-wold:ENC4_cDNA685'], 'schema_version': '26', 'status': 'in progress', 'lab': '/labs/barbara-wold/', 'award': '/awards/UM1HG009443/', 'date_created': '2022-03-28T20:24:54.440473+00:00', 'submitted_by': '/users/bc5b62f7-ce28-4a1e-b6b3-81c9c5a86d7a/', 'documents': [], 'references': [], 'source': '/sources/rush-university/', 'biosample_ontology': '/biosample-types/tissue_UBERON_0006483/', 'genetic_modifications': [], 'alternate_accessions': [], 'description': 'human_brain_Rush ID_E3348003_BS123_ENC4_cDNA685', 'treatments': [], 'dbxrefs': [], 'donor': '/human-donors/ENCDO730UYW/', 'organism': '/organisms/human/', 'internal_tags': ['RushAD'], 'disease_term_id': ['DOID:10652'], '@id': '/biosamples/ENCBS745ELX/', '@type': ['Biosample', 'Item'], 'uuid': '01b97ed1-4636-4022-ad43-4676aeb96db2', 'sex': 'female', 'age': '90 or above', 'age_units': 'year', 'disease_term_name': ["Alzheimer's diseas

# Lookup clinical data

In [ ]:
biosample